In [9]:
import pandas as pd

filepath_dict = { 'yelp': '/content/sample_data/data/yelp_labelled.txt',
                  'amazon': '/content/sample_data/data/amazon_cells_labelled.txt',
                  'imdb': '/content/sample_data/data/imdb_labelled.txt'}

df_list = []

for source, filepath in filepath_dict.items():
  df = pd.read_csv(filepath, names = ['sentence', 'label'], sep ='\t')
  df['source'] = source
  df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0])  

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object


In [10]:
from sklearn.model_selection import train_test_split

df_yelp = df[df['source'] == 'yelp']

sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences,y,test_size = 0.25, random_state = 1000
)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)


In [12]:
X_train

<750x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 7368 stored elements in Compressed Sparse Row format>

In [13]:
X_test

<250x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 2069 stored elements in Compressed Sparse Row format>